# VacationPy
----


In [70]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import json

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame

In [71]:
# Load the csv exported in WeatherPy to a DataFrame
weatherdata = pd.read_csv("Output/weather_data.csv")
weatherdata.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,saint-philippe,90,RE,1580008370,83,-21.36,55.77,75.20,6.93
1,punta arenas,75,CL,1580008363,70,-53.15,-70.92,48.20,25.28
2,bredasdorp,68,ZA,1580008363,87,-34.53,20.04,59.00,13.87
3,atuona,74,PF,1580008374,77,-9.80,-139.03,80.74,13.80
4,balikpapan,40,ID,1580008846,83,-1.27,116.83,84.20,4.70


### Humidity Heatmap

In [73]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Use the Lat and Lng as locations and Humidity as the weight.
locations = weatherdata[["Lat", "Lng"]]
humidity = weatherdata["Humidity"].astype(float)

# Plot Heatmap
fig = gmaps.figure(map_type="TERRAIN")

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=weatherdata["Humidity"].max(),
                                 point_radius=1)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions:
** A max temperature lower than 80 degrees but higher than 70.
** Wind speed less than 10 mph.
** Zero cloudiness.
* Drop rows with null values.

In [114]:
weatherdata[["Max Temp","Wind Speed","Cloudiness"]].astype(float)
ideal_weather = weatherdata.loc[(weatherdata["Max Temp"] < 80)] 
ideal_weather = ideal_weather.loc[(weatherdata["Max Temp"] > 70)]
ideal_weather = ideal_weather.loc[(weatherdata["Wind Speed"] < 10)]
ideal_weather = ideal_weather.loc[(weatherdata["Cloudiness"] == 0)]
hotel_df = ideal_weather.dropna().set_index("City").reset_index()
hotel_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,arraial do cabo,0,BR,1580008713,82,-22.97,-42.03,75.16,6.76
1,tarauaca,0,BR,1580008859,87,-8.16,-70.77,74.12,2.10
2,alacranes,0,CU,1580009075,78,22.77,-81.57,71.60,9.17
3,young,0,UY,1580009090,80,-32.68,-57.63,78.01,5.17
4,kpandae,0,GH,1580009105,11,8.47,-0.01,71.20,3.47
5,santa maria,0,BR,1580009115,69,-29.68,-53.81,73.40,1.12
6,curico,0,CL,1580009117,31,-34.98,-71.23,77.00,2.24
7,paredon,0,MX,1580009119,71,16.03,-93.87,78.49,3.33
8,san lorenzo,0,PY,1580009120,73,-25.33,-57.53,77.00,3.67


### There are 9 cities with my ideal locations

### Hotel Map
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [115]:
hotel_df["Hotel Name"] = ""

In [116]:
params = {
    "radius": 5000,
    "keyword": "hotel",
    "key": g_key
}
for index,row in hotel_df.iterrows():
    # get lat, lng from df
    lng = row["Lng"]
    lat = row["Lat"]
    
    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    hotel_name = requests.get(base_url, params=params)
    
#     print the hotel_name url, avoid doing for public github repos in order to avoid exposing key
#     print(name_address.url)

    # convert to json
    hotel_name = hotel_name.json()
    
    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_name["results"][0]["name"]
    except (KeyError, IndexError):
        pass
        
hotel_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,arraial do cabo,0,BR,1580008713,82,-22.97,-42.03,75.16,6.76,Hotel da Canoa
1,tarauaca,0,BR,1580008859,87,-8.16,-70.77,74.12,2.10,Hotel Ferreira
2,alacranes,0,CU,1580009075,78,22.77,-81.57,71.60,9.17,
3,young,0,UY,1580009090,80,-32.68,-57.63,78.01,5.17,Apartamento Young
4,kpandae,0,GH,1580009105,11,8.47,-0.01,71.20,3.47,
5,santa maria,0,BR,1580009115,69,-29.68,-53.81,73.40,1.12,Itaimbé Palace
6,curico,0,CL,1580009117,31,-34.98,-71.23,77.00,2.24,Radisson Hotel Curico
7,paredon,0,MX,1580009119,71,16.03,-93.87,78.49,3.33,
8,san lorenzo,0,PY,1580009120,73,-25.33,-57.53,77.00,3.67,Sheraton Asuncion Hotel


In [117]:
# Remove rows with empty cells
hotel_df = hotel_df.loc[(hotel_df["Hotel Name"] != "")].set_index("City").reset_index()
hotel_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,arraial do cabo,0,BR,1580008713,82,-22.97,-42.03,75.16,6.76,Hotel da Canoa
1,tarauaca,0,BR,1580008859,87,-8.16,-70.77,74.12,2.10,Hotel Ferreira
2,young,0,UY,1580009090,80,-32.68,-57.63,78.01,5.17,Apartamento Young
3,santa maria,0,BR,1580009115,69,-29.68,-53.81,73.40,1.12,Itaimbé Palace
4,curico,0,CL,1580009117,31,-34.98,-71.23,77.00,2.24,Radisson Hotel Curico
5,san lorenzo,0,PY,1580009120,73,-25.33,-57.53,77.00,3.67,Sheraton Asuncion Hotel


In [118]:
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [119]:
# Add marker layer ontop of heat map
fig = gmaps.figure(map_type = "TERRAIN")
markers = gmaps.marker_layer(
    locations,
    info_box_content = hotel_info
)
fig.add_layer(markers)
fig.add_layer(heat_layer)
fig
# Display Map

Figure(layout=FigureLayout(height='420px'))

## Most of the hotels with my ideal vacation conditions are in South America